# Problem setting
* Using the measurments taken from  scada System concerning the wind turbins as wind speed, wind direction , generated wind power and the theoritical wind power for the time interval of 10 min, then all sum up to one hour, we are seeking the prediction of the generated wind power for every time interval of one hour.
## Content
#### the Data contains the following five columns:
* Date/Time: date and time at which the measurments were taken
* LV ActivePower (kw): the generated power by the turbine in Kilowatt unit
* Theoretical_Power_Curve (KWh): the theoretical generated power by the turbine given by the manufacturer
* Wind Speed (m/s): the wind speed in meter/second
* Wind direction (°): Wind direction in degree



In [10]:
### importing libraries
import io
from IPython.display import display
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
### plotly library
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
### debugging purpose
import pdb
## keras library
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Masking,TimeDistributed
from keras.callbacks import EarlyStopping 
### sklearn 
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [11]:
##setting the right directory
current_dir = os.getcwd()
reg = r'(.*\\Final\s*Project)(.+)'
current_dir = re.sub(reg,r"\1",current_dir)
new_dir = current_dir +'\\Wind_Turbine_Scada'
os.chdir(new_dir)

In [12]:
##load the data
Turbine = pd.read_csv('T1.csv',sep = ",")
Turbine = Turbine.round(2)
## convert the Date/Time column to datetime type
Turbine['Date/Time'] = pd.to_datetime(Turbine['Date/Time'],format = "%d %m %Y %H:%M")
## Group the date by Date/Time and sum up all the records up to one hour
Turbine = Turbine.groupby(pd.Grouper(key = "Date/Time",freq = 'H')).sum()
## display the first 5 records of the data
display(Turbine.head())
## show useful info about the data 
display(f"Infos about DataFrame: \n {Turbine.info()}")

,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
Date/Time,,,,
2018-01-01 00:00:00,2342.89,33.04,2834.42,1602.70
2018-01-01 01:00:00,2763.23,33.85,3080.51,1553.66
2018-01-01 02:00:00,4401.93,38.72,4730.40,1610.37
2018-01-01 03:00:00,5456.17,40.88,5692.76,1536.09
2018-01-01 04:00:00,8361.12,46.50,8438.36,1485.92


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2018-01-01 00:00:00 to 2018-12-31 23:00:00
Freq: H
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   LV ActivePower (kW)            8760 non-null   float64
 1   Wind Speed (m/s)               8760 non-null   float64
 2   Theoretical_Power_Curve (KWh)  8760 non-null   float64
 3   Wind Direction (°)             8760 non-null   float64
dtypes: float64(4)
memory usage: 342.2 KB


'Infos about DataFrame: \n None'

In [7]:
## check for missing values
Turbine.isnull().sum(axis = 0)

LV ActivePower (kW)              0
Wind Speed (m/s)                 0
Theoretical_Power_Curve (KWh)    0
Wind Direction (°)               0
dtype: int64

here you should emphasize in which year season the production increases !
and if it depends on the wind speed ?

- 

In [13]:
### displays the data distribution
Turbine_vis = Turbine.groupby(pd.Grouper(key = None,freq = 'D')).mean()
## define the title of each plot
titles = ['ActivePower vs Date',
          'Wind speed vs Date',
          'Theoritical power vs Date',
          'Wind direction vs Date']
## name of each curve plot curve
names = ['ActivePower',
         'Wind Speed',
         'Theoritical power',
         'Wind direction']
## create a canvas with 4 subplots
fig = make_subplots(rows = 2,cols =2,subplot_titles = titles,
                    horizontal_spacing=0.2,vertical_spacing = 0.3)
## loop over the subplots to plot each column in function of Date/Time 
for i in range(4):
    if i <= 1:
        fig.add_trace(go.Scatter(x =Turbine_vis.index,
                                 y = Turbine_vis.iloc[:,i],name = names[i]),
                                 row =1,col=i+1)
        fig.update_xaxes(title_text = "Date/Time",row =1,col = i+1)
        fig.update_yaxes(title_text = list(Turbine_vis)[i],row =1,col= i+1)
    else:
        fig.add_trace(go.Scatter(x =Turbine_vis.index,y = Turbine_vis.iloc[:,i],
                                 name = names[i]),row =2,
                                 col=i-1)
        fig.update_xaxes(title_text = "Date/Time", row = 2,col = i-1)
        fig.update_yaxes(title_text = list(Turbine_vis)[i],row = 2, col = i-1)
        
fig.update_layout(width = 850, height = 600)
fig.show()

#### Correlation between dependent and independent variables ('LV Activepower vs others') 
* from the plot it's clear that wind speed and theoretical_power_curve are highly correlated with the dependent variable ('LV Activepower'), however the wind direction seems to be less correlated with the target variable.

In [15]:
## define the title of each plot
titles = ['Correlation between ActivePower and Wind speed',
          'Correlation between ActivePower and Theoritical power',
          'Correlation between ActivePower and wind direction']
## the name of each plot curve 
names = ['ActivePower vs Wind speed',
         'ActivePower vs Theoritical power',
         'ActivePower vs Wind direction']
## creat a canvas with three subplots
fig = make_subplots(rows = 3, cols=1,subplot_titles= titles,vertical_spacing = 0.25)
## loop over the subplots to plot the correlation between dependent variable ('LV Activ') and the indepedent ones 
for i in range(3):
    fig.add_trace(go.Scatter(y = Turbine_vis['LV ActivePower (kW)'],
                             x = Turbine_vis.iloc[:,i+1],name = names[i],                           
                             mode = 'markers',
                             marker = {
                                     'size':5,
                                     'line':{
                                             'width': 1
                                            }
                                     }),
                                     row = i+1,
                                     col = 1)
    fig.update_yaxes(title = "ActivePower(KWh)",row = i+1,col = 1)
    fig.update_xaxes(title = list(Turbine_vis.columns)[i+1], row = i+1, col = 1)

fig.update_layout(width = 900, height = 600)
fig.show()

In [16]:
## split the data 
def split_data(data,split_rate = 0.3):
    '''
    splits the data using the specified split rate 
    '''
    idx = 1-int(len(data)*split_rate)
    X_test = data.iloc[idx:,:]
    X_train = data.iloc[:idx,:]
    return X_train,X_test

train,test = split_data(Turbine)

In [17]:
### Features scaling
def feature_scaling(X_train,X_test,standardization = False,minmax =False):
    '''
    scale the data by using either min max scaling or standardization
    '''
    if standardization:
        SCALER = StandardScaler()
        scaler = SCALER.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    elif minmax:
        SCALER = MinMaxScaler(feature_range=(-1,1))
        scaler = SCALER.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    return X_train,X_test,scaler
### scaling the columns using min max scaler
X_train,X_test,scaler = feature_scaling(train,test,minmax = True)

In [18]:
## create new shape of data with specific timestep (,timestep,features)
def create_lookback(X_train,X_test,timestep):

    X_test = np.concatenate((X_train[-timestep:],X_test),axis = 0) ### to predict the first timestep instances in test set

    X_train_lstm = []
    X_test_lstm = []
    y_train_lstm = []
    y_test_lstm = []
    for i in range(timestep,len(X_train)):
        X_train_lstm.append(X_train[i-timestep:i,:])
        y_train_lstm.append(X_train[i,0])
    for i in range(timestep,len(X_test)):
        X_test_lstm.append(X_test[i-timestep:i,:])
        y_test_lstm.append(X_test[i,0])

    X_train_lstm,X_test_lstm = np.array(X_train_lstm),np.array(X_test_lstm)
    y_train_lstm,y_test_lstm = np.array(y_train_lstm),np.array(y_test_lstm)
    return X_train_lstm,X_test_lstm,y_train_lstm,y_test_lstm

timestep = 300

X_train_lstm,X_test_lstm,y_train_lstm,y_test_lstm = create_lookback(X_train,X_test,timestep)

## Implementation of  Recurrent Neural Network model (LSTM version)

In [19]:
def build_model(X_train,y_train,epochs =5,dropout = 0.2,batch = 14):

    early_stopping = EarlyStopping(patience=7, monitor='loss')
    opt = keras.optimizers.Adam(learning_rate=0.0005)
    reg = Sequential()
    reg.add(LSTM(units = 52,dropout=dropout,return_sequences= True,
                 input_shape =(X_train.shape[1],X_train.shape[2]),name = 'layer_1'))
    reg.add(LSTM(units = 32,dropout = dropout))
    reg.add(Dense(1))
    ### compiling the RNN 
    reg.compile(optimizer =opt, loss = 'mse')
    reg.summary()
    history = reg.fit(x = X_train,y = y_train,epochs =epochs,shuffle=False,verbose=1,
          validation_split = 0.2, batch_size = batch,callbacks=[early_stopping])
    return reg,history

reg,history = build_model(X_train_lstm,y_train_lstm, epochs = 10,batch = 30)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_1 (LSTM)               (None, 300, 52)           11856     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10880     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 22,769
Trainable params: 22,769
Non-trainable params: 0
_________________________________________________________________
Train on 4666 samples, validate on 1167 samples
Epoch 1/10
4666/4666 [==============================] - 91s 19ms/step - loss: 0.2786 - val_loss: 0.1633
Epoch 2/10
4666/4666 [==============================] - 81s 17ms/step - loss: 0.1930 - val_loss: 0.1346
Epoch 3/10
4666/4666 [==============================] - 93s 20ms/step - loss: 0.1661 - val_loss: 0.1112
Epoch 4/10
4666/4666

In [86]:
##evaluation
steps = 30
pred = reg.predict(X_test_stp)
Turbine_pred = pd.concat([X_test_date.iloc[steps:].reset_index(drop = True),
               pd.Series(data = pred.reshape((pred.shape[0])), name = 'Predicted Activpower'),
               pd.Series(data = y_test_stp, name = "Activpower")],axis = 1)
Turbine_pred_vis = Turbine_pred.groupby(pd.Grouper(key = "Date/Time",freq = 'D')).mean()

In [92]:
## plot the result 
fig = go.Figure()
fig.add_trace(
    go.Scatter(y = Turbine_pred_vis['Activpower'],
              x = Turbine_pred_vis.index)
)
fig.add_trace(
    go.Scatter(y = Turbine_pred_vis['Predicted Activpower'],
              x = Turbine_pred_vis.index)
)
fig.show()